In [1]:
import copy
import multiprocessing as mp

import numpy as np
import gymnasium as gym

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import games

In [14]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # common layers
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        # action policy layers
        self.act_conv1 = nn.Conv2d(128, 4, kernel_size=1)
        self.act_fc1 = nn.Linear(4*3*3, 3*3)
        # state value layers
        self.val_conv1 = nn.Conv2d(128, 4, kernel_size=1)
        self.val_fc1 = nn.Linear(4*3*3, 1)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x_act = F.relu(self.act_conv1(x))
        x_act = x_act.view(-1, 4*3*3)
        x_act = F.softmax(self.act_fc1(x_act), dim=-1)
        x_val = F.relu(self.val_conv1(x))
        x_val = x_val.view(-1, 4*3*3)
        x_val = self.val_fc1(x_val)
        return x_act, x_val
    
class PolicyValueNet:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.net = Net().to(self.device)
        self.lr = 1e-3
        self.c = 1e-4
        self.optimizer = optim.Adam(self.net.parameters(), lr=self.lr, weight_decay=self.c)
    
    def policy_value(self, state):
        state = torch.as_tensor(state, dtype=torch.float32, device=self.device)
        with torch.no_grad():
            action_p, value = self.net(state)
        action_p = action_p.cpu().numpy()
        value = value.cpu().numpy()
        return action_p, value
    
    def update(self, state, mcts_p, reward):
        state = torch.tensor(state, dtype=torch.float32, device=self.device)
        mcts_p = torch.tensor(mcts_p, dtype=torch.float32, device=self.device)
        reward = torch.tensor(reward, dtype=torch.float32, device=self.device)
        self.optimizer.zero_grad()
        action_p, value = self.net(state)
        # cross entorpy loss for the search probabilities 
        policy_loss = torch.mean(torch.sum(mcts_p * torch.log(action_p), -1))
        # mse loss for the state value
        value_loss = F.mse_loss(value.view(-1), reward)
        # total loss
        loss = value_loss - policy_loss
        loss.backward()
        self.optimizer.step()
        return value_loss.item(), -1 * policy_loss.item()
            
    def save(self, filename):
        torch.save(self.net.state_dict(), filename)
    
    @classmethod
    def load(cls, filename):
        policy_value_net = cls()
        policy_value_net.net.load_state_dict(torch.load(filename))
        return policy_value_net

In [15]:
class AgentNode:
    """
    Agent node of MCTS
    """
    def __init__(self, parent, action, num_actions, P, N=0, W=0):
        """
        Each state_action pair (s, a) stores a set of statistics, {N(s, a), W(s, a), Q(s, a), P(s, a)},
        where N(s, a) is the visit count, W(s, a) is the total action-value, Q(s, a) is the mean action-value,
        and P(s, a) is the prior probability of selecting a in s.
        """
        self.parent = parent
        self.action = action
        self.num_actions = num_actions
        self.P = P
        self.N = N
        self.W = W
        self.children = {}
        self.child_N = np.zeros(num_actions, dtype=np.float32)
        self.child_W = np.zeros(num_actions, dtype=np.float32)
        self.child_P = None
        self.agent_index = None
        self.is_expanded = False

    def select(self, c_puct_base, c_puct_init):
        """

        Args:
            c_puct_base (_type_): _description_
            c_puct_init (_type_): _description_

        Returns:
            _type_: _description_
        """
        c_puct = np.log((1 + self.N + c_puct_base) / c_puct_base) + c_puct_init
        Q = self.child_W / np.where(self.child_N > 0, self.child_N, 1)
        U = c_puct * self.child_P * np.sqrt(self.N) / (1 + self.child_N)
        action = np.argmax(-1 * Q + U)
    
        if action not in self.children.keys():
            self.children[action] = AgentNode(
                self,
                action,
                self.num_actions,
                self.child_P[action]
            )
        return action, self.children[action]
    
    def expand(self, agent_index, next_P):
        self.agent_index = agent_index
        self.child_P = next_P
        self.is_expanded = True

    def back_propagate(self, value):
        self.N += 1
        self.W += value
        if self.parent:
            self.parent.child_N[self.action] = self.N
            self.parent.child_W[self.action] = self.W
            self.parent.back_propagate(-value)
            
    def as_root(self):
        self.parent = None
        self.action = None
        self.P = 1
    
    @property
    def Q(self):
        return self.W / self.N

In [16]:
class MCTSPlayer:
    def __init__(self, policy_value_net, c_puct_base=5, c_puct_init=0, num_simulations=100, noise=False, deterministic=False):
        self.policy_value_net = policy_value_net
        self.c_puct_base = c_puct_base
        self.c_puct_init = c_puct_init
        self.num_simulations = num_simulations
        self.noise = noise
        self.deterministic = deterministic
        self.rng = np.random.default_rng()
    
    def to_state(self, observation, info, agent_mark_mapping):
        """ 
        Transfer environmental observation and information to a state tensor as neural network input.
        Board observation will transfer to an N * N * M image stack, each plane represent the board positions 
        oriented to a certain player (N * N with dummy), current player's plane is on the top.
        state: numpy array with shape (2, 3, 3)
        """
        agent_index = info['agent_index']
        mark_list = list(agent_mark_mapping.values())
        num_agents = len(mark_list)
        array_list = []
        for i in range(num_agents):
            index = (agent_index + i) % num_agents
            mark = mark_list[index]
            array_list.append(observation == mark)
        state = np.stack(array_list, dtype=np.float32)
        return state
    
    def add_dirchleet_noise(self, node, action_space, epsilon=0.25, alpha=0.03):
        alphas = np.ones(action_space.n) * alpha
        noise = self.rng.dirichlet(alphas)
        node.child_P = node.child_P * (1 - epsilon) + noise * epsilon
        
    def get_mcts_p(self, child_N, temperature=0.1):
        child_N = child_N ** (1 / temperature)
        return child_N / sum(child_N)
    
    def mcts(self, env, observation, info, root_node=None):
        # Initialize environment and root node.
        action_space = env.unwrapped.action_space
        agent_mark_mapping = env.unwrapped.agent_mark_mapping
        root_state = self.to_state(observation, info, agent_mark_mapping)
        prior_p, value = self.policy_value_net.policy_value(root_state)
        if not root_node:
            root_node = AgentNode(None, None, action_space.n, 1)
        root_node.expand(info['agent_index'], prior_p[0])
        root_node.back_propagate(value.item())
        
        # Start mcts simulation.
        while root_node.N < self.num_simulations:
            sim_env = copy.deepcopy(env)
            node = root_node
            # Add dirchleet noise.
            if self.noise:
                self.add_dirchleet_noise(node, action_space)
                
            while node.is_expanded:
                # SELECT
                action, node = node.select(self.c_puct_base, self.c_puct_init)
                # INTERACT
                observation, reward, terminated, truncated, info = sim_env.step(action)               
                if terminated or truncated:
                    # BACK PROPAGATE (REWARD)
                    node.back_propagate(-reward)
            # EVALUATE
            state = self.to_state(observation, info, agent_mark_mapping)
            prior_p, value = self.policy_value_net.policy_value(state)
            # EXPAND
            node.expand(info['agent_index'], prior_p[0])
            # BACK PROPAGATE (VALUE)
            node.back_propagate(value.item())
        
        # Choose best action for root node (deterministic or stochastic).
        mcts_p = self.get_mcts_p(root_node.child_N)
        if self.deterministic:
            action = np.argmax(mcts_p)
        else:
            action = self.rng.choice(np.arange(action_space.n), p=mcts_p)
        
        if action in root_node.children.keys():
            next_root_node = root_node.children[action]
            next_root_node.as_root()
        else:
            next_root_node = None
        
        return root_state, action, mcts_p, next_root_node

In [17]:
# env = gym.make('games/TicTacToe', max_episode_steps=100)
# policy_value_net = PolicyValueNet()
# player = MCTSPlayer(policy_value_net)
# observation, info = env.reset()
# root_state, action, mcts_p, next_root_node = player.mcts(env, observation, info)

In [18]:
def selfplay(env, policy_value_net):
    agent_index_list = []
    state_list = []
    mcts_p_list = []
    
    
    observation, info = env.reset()
    player = MCTSPlayer(policy_value_net)
    root_node = None
    is_end = False
    
    while not is_end:
        agent_index_list.append(info['agent_index'])
        root_state, action, mcts_p, root_node = player.mcts(env, observation, info, root_node)
        state_list.append(root_state)
        mcts_p_list.append(mcts_p)
        observation, reward, terminated, truncated, info = env.step(action)
        is_end = terminated or truncated

    reward_list = [
        reward if index == agent_index_list[-1] else -reward for index in agent_index_list
    ]
    
    return np.array(state_list), np.array(mcts_p_list), np.array(reward_list)

In [19]:
env = gym.make('games/TicTacToe', max_episode_steps=100)
policy_value_net = PolicyValueNet()
state_list, mcts_p_list, value_list = selfplay(env, policy_value_net)

In [22]:
env = gym.make('games/TicTacToe', max_episode_steps=100)
policy_value_net = PolicyValueNet()
num_episodes = 100
for _ in range(num_episodes):
    state, mcts_p, reward = selfplay(env, policy_value_net)
    value_loss, policy_loss = policy_value_net.update(state, mcts_p, reward)
    print(f'value loss: {value_loss}, policy loss: {policy_loss}')
policy_value_net.save('weights.pth')

value loss: 0.9995370507240295, policy loss: 2.156324863433838
value loss: 1.01210618019104, policy loss: 2.1506924629211426
value loss: 0.9906719923019409, policy loss: 2.1616837978363037
value loss: 0.9546806216239929, policy loss: 2.1479806900024414
value loss: 0.9881836175918579, policy loss: 2.1363799571990967
value loss: 1.010827898979187, policy loss: 2.1650583744049072
value loss: 1.0745843648910522, policy loss: 2.165300130844116
value loss: 0.8930301070213318, policy loss: 2.179348945617676
value loss: 0.8937676548957825, policy loss: 2.1887400150299072
value loss: 1.1154314279556274, policy loss: 2.1826398372650146
value loss: 1.0333322286605835, policy loss: 2.154114007949829
value loss: 1.0666056871414185, policy loss: 2.165682315826416
value loss: 0.976568341255188, policy loss: 2.1685702800750732
value loss: 0.8880721926689148, policy loss: 2.1961989402770996
value loss: 0.9891560077667236, policy loss: 2.174851894378662
value loss: 0.9904701113700867, policy loss: 2.186

In [23]:
player = MCTSPlayer(policy_value_net)
env = gym.make('games/TicTacToe', max_episode_steps=100, render_mode='rgb_array')

In [46]:
observation, info = env.reset()
print(observation)

[[0 0 0]
 [0 0 0]
 [0 0 0]]


In [47]:
info['agent_index']

0

In [48]:
root_state, action, mcts_p, next_root_node = player.mcts(env, observation, info)
print(action)

5


In [49]:
observation, reward, terminated, truncated, info = env.step(action)

In [50]:
observation

array([[0, 0, 0],
       [0, 0, 2],
       [0, 0, 0]], dtype=int8)

In [38]:
observation, reward, terminated, truncated, info = env.step(4)

In [39]:
root_state, action, mcts_p, next_root_node = player.mcts(env, observation, info)
print(action)

5


In [40]:
observation, reward, terminated, truncated, info = env.step(5)

In [41]:
observation

array([[0, 0, 0],
       [0, 2, 1],
       [0, 0, 0]], dtype=int8)

In [52]:
def to_state(observation, info, agent_mark_mapping):
    agent_index = info['agent_index']
    # mark = agent_mark_mapping[info['agent_index']]
    mark_list = list(agent_mark_mapping.values())
    num_agents = len(mark_list)
    array_list = []
    for i in range(num_agents):
        index = (agent_index + i) % num_agents
        mark = mark_list[index]
        array_list.append(observation == mark)
    state = np.stack(array_list)
    return state

In [53]:
def add_dirchleet_noise(node, action_space, epsilon=0.25, alpha=0.03):
    alphas = np.ones(action_space.n) * alpha
    noise = np.random.default_rng().dirichlet(alphas)
    node.child_P = node.child_P * (1 - epsilon) + noise * epsilon

In [54]:
def get_mcts_p(child_N, temperature=0.1):
    child_N = child_N ** (1 / temperature)
    return child_N / sum(child_N)

In [55]:
def mcts(env, net, c_puct_base=5, c_puct_init=0, num_simulations=100, noise=False, deterministic=False):
    # Initialize environment and root node.
    action_space = env.unwrapped.action_space
    agent_mark_mapping = env.unwrapped.agent_mark_mapping
    root_node = AgentNode(None, None, action_space.n, 1)
    observation, info = env.reset()
    state = to_state(observation, info, agent_mark_mapping)
    prior_p, value = net.policy_value_eval(state)
    root_node.expand(info['agent_index'], prior_p[0])
    root_node.back_propagate(value[0])
    
    # Start mcts simulation.
    while root_node.N < num_simulations:
        sim_env = copy.deepcopy(env)
        node = root_node
        is_end = False
        while not is_end:
            # SELECT
            action, node = node.select(c_puct_base, c_puct_init)
            # INTERACT
            observation, reward, terminated, truncated, info = sim_env.step(action)
            state = to_state(observation, info, agent_mark_mapping)
            # EVALUATE
            prior_p, value = net.policy_value_eval(state)
            # EXPAND
            node.expand(info['agent_index'], prior_p[0])
            # BACK PROPAGATE
            node.back_propagate(value.item())
            is_end = terminated or truncated
    
    # Choose best action for root node (deterministic or stochastic).
    mcts_p = get_mcts_p(root_node.child_N)
    if deterministic:
        action = np.argmax(mcts_p)
    else:
        action = np.random.default_rng().choice(np.arange(action_space.n), p=mcts_p)
    
    return action, mcts_p, root_node.Q

In [115]:
env = gym.make('games/TicTacToe', max_episode_steps=100)
policy_value_net = PolicyValueNet()

action_space = env.unwrapped.action_space
agent_mark_mapping = env.unwrapped.agent_mark_mapping

observation, info = env.reset()
root_node = AgentNode(
    parent = None,
    action = None,
    num_actions = action_space.n,
    P = 1
)

state = to_state(observation, info, agent_mark_mapping)
prior_p, value = policy_value_net.policy_value_eval(state)
root_node.expand(info['agent_index'], prior_p[0])
root_node.back_propagate(value.item())


num_simulations = 10000
c_puct_base=5
c_puct_init=0


sim_env = copy.deepcopy(env)
while root_node.N < num_simulations:
    sim_env = copy.deepcopy(env)
    node = root_node
    while node.is_expanded:
        # SELECT
        action, node = node.select(c_puct_base, c_puct_init)
        # INTERACT
        observation, reward, terminated, truncated, info = sim_env.step(action)               
        if terminated or truncated:
            # BACK PROPAGATE (REWARD)
            node.back_propagate(reward)
    # EVALUATE
    state = to_state(observation, info, agent_mark_mapping)
    prior_p, value = policy_value_net.policy_value_eval(state)
    # EXPAND
    node.expand(info['agent_index'], prior_p[0])
    # BACK PROPAGATE (VALUE)
    node.back_propagate(value.item())

print(root_node.child_W)

[6. 1. 4. 9. 3. 0. 3. 4. 3.]


In [116]:
root_node.Q

-0.0033